# IMPORTAÇÃO DE BIBLIOTECAS E DRIVE

As 4 primeiras linhas foram adicionadas antes de saber o que seria necessário durante a execução do TCC. As seguintes foram acrescentadas assim que necessário.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.linear_model import ElasticNetCV

Montagem do Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# IMPORTAÇÃO DE DADOS

In [3]:
df_pre = pd.read_csv("/content/drive/MyDrive/tcc/dataframes/df_pre.csv")

# EXPLORAÇÃO DE DADOS

In [4]:
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   codigo                5570 non-null   int64  
 1   aids                  5570 non-null   int64  
 2   animais               5570 non-null   int64  
 3   dengue                5570 non-null   int64  
 4   hanseniase            5570 non-null   int64  
 5   hepatites             5570 non-null   int64  
 6   sadquirida            5570 non-null   int64  
 7   scongenita            5570 non-null   int64  
 8   sgestante             5570 non-null   int64  
 9   toxico                5570 non-null   int64  
 10  tuberculose           5570 non-null   int64  
 11  autoprovocada_sim     5570 non-null   int64  
 12  autoprovocada_não     5570 non-null   int64  
 13  violencia             5570 non-null   int64  
 14  estabelecimentos      5570 non-null   int64  
 15  profissionais        

In [5]:
df_pre["estado"] = df_pre["estado"].astype("category")
df_pre["metropolitana"] = df_pre["metropolitana"].astype("category")
df_pre["amazonia"] = df_pre["amazonia"].astype("category")
df_pre["semiarido"] = df_pre["semiarido"].astype("category")
df_pre["regiao"] = df_pre["regiao"].astype("category")

In [6]:
df_pre.head(100)

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,impostos,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil
0,110001,1,192,833,51,41,28,2,12,186,...,29351.0,495629.0,21600.73,22945,-11.92870,-61.9958,N,1828,659,23
1,110037,1,199,90,86,12,13,1,2,31,...,11248.0,288681.0,21802.07,13241,-12.14050,-61.8379,N,803,252,4
2,110040,5,62,65,43,66,18,2,11,18,...,18150.0,358464.0,16728.75,21428,-9.71470,-63.3193,N,1154,369,24
3,110034,5,78,126,24,8,16,1,7,35,...,10423.0,260348.0,18065.91,14411,-11.34670,-62.2852,N,911,414,8
4,110002,121,252,198,224,240,296,8,89,321,...,284398.0,2578830.0,23908.38,107863,-9.90612,-63.0330,N,8580,2557,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,130200,5,57,2,12,1,5,0,6,1,...,4646.0,129557.0,14162.35,9148,-2.74119,-58.0297,N,844,185,9
96,130210,3,39,17,4,4,5,2,5,1,...,1378.0,64130.0,23277.54,2755,-1.88274,-66.9295,N,752,77,15
97,130230,6,141,3,33,75,55,0,16,0,...,6188.0,227757.0,15908.16,14317,-2.74904,-66.7696,N,3190,336,71
98,130240,10,312,60,71,79,78,9,41,5,...,18852.0,506494.0,10994.24,46069,-7.26453,-64.7952,N,3723,776,75


Sífilis adquirida não inclui sífilis em gestante porque alguns valores em gestante é maior do que adquirida. Explorei até achar no index 6 (codigo 110003) e para confirmar index 16 (codigo 110008).

In [7]:
df_sif = df_pre

Criação de um indicador chamado "squalidade", que representa a razão entre o total de casos de sífilis adquirida e o número de casos de sífilis em gestante. A ideia é que quanto maior for o valor da razão melhor é a a notificação naquela cidade, considerando que o número de casos totais em homens e mulheres não grávidas deveria ser muito maior do que o número de casos confirmados em mulheres grávidas por simples exercício de lógica.

In [8]:
df_sif["squalidade"] = df_sif["sadquirida"]/df_sif["sgestante"]

In [9]:
df_sif["squalidade"]

0       2.333333
1       6.500000
2       1.636364
3       2.285714
4       3.325843
          ...   
5565    0.000000
5566    0.500000
5567    0.000000
5568    2.333333
5569    0.000000
Name: squalidade, Length: 5570, dtype: float64

In [10]:
df_ordenado = df_sif.sort_values(by=["squalidade"], axis=0, ascending=False)
df_ordenado[["squalidade", "sadquirida", "sgestante"]]

,squalidade,sadquirida,sgestante
4798,inf,2,0
732,inf,12,0
3722,inf,4,0
718,inf,1,0
3721,inf,2,0
...,...,...,...
5539,NaN,0,0
5542,NaN,0,0
5544,NaN,0,0
5552,NaN,0,0


**Explorei a tabela deste indicador de qualidade em ordem descrescente e percebi que dos 5570 municípios, já no 864º município, o valor da razão seria de 5 e no 1939º esse valor seria de 2.5, valores estes que são absolutamente incompatíveis com qualquer sistema de notificação que esteja funcionando. Não faz sentido acreditar que em 84% dos municípios brasileiros, para cada 3 casos de sífilis em mulheres teremos 1 em grávida e 2 em não grávidas. Ainda pior, em 65% dos municípios brasileiros, teríamos mais casos de sífilis em mulheres gestantes do que em mulheres não gestantes. Considero aqui que a razão entre os casos notificados de sífilis em homens e mulheres não grávidas é 3:2 (pesquisei no DATASUS, valor aproximado). Desconsidero nesta análise que o número de casos de sífilis em grávidas inclui somente casos confirmados, o que só contribui para aumentar a certeza da baixa qualidade dos dados. Este é o momento mais crítico do trabalho porque comprova a principal hipótese do mesmo, a de que temos problemas muito graves em notificações de doenças e agravos. Continuo a exploração dos dados de sífilis mas considero que não é necessário continuar explorando mais doenças ou agravos pois nenhuma delas teria uma forma de validar interna tão forte quanto a desta doença que separou grávidas de não grávidas. Ainda, os modelos de machine learning serão realizados, mas fica claro que devem ser interpretados com extrema cautela ou nem interpretados.**

Quantificados os valores infinitos e nulos.

In [11]:
df_sif[df_sif["squalidade"] == float("inf")]["squalidade"].count()

382

In [12]:
df_sif["squalidade"].isna().sum()

187

Retirados valores infinitos e nulos

In [13]:
df_sif.replace([np.inf, -np.inf], np.nan, inplace=True)

In [14]:
df_sif = df_sif.dropna()

In [15]:
df_sif

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil,squalidade
0,110001,1,192,833,51,41,28,2,12,186,...,495629.0,21600.73,22945,-11.92870,-61.9958,N,1828,659,23,2.333333
1,110037,1,199,90,86,12,13,1,2,31,...,288681.0,21802.07,13241,-12.14050,-61.8379,N,803,252,4,6.500000
2,110040,5,62,65,43,66,18,2,11,18,...,358464.0,16728.75,21428,-9.71470,-63.3193,N,1154,369,24,1.636364
3,110034,5,78,126,24,8,16,1,7,35,...,260348.0,18065.91,14411,-11.34670,-62.2852,N,911,414,8,2.285714
4,110002,121,252,198,224,240,296,8,89,321,...,2578830.0,23908.38,107863,-9.90612,-63.0330,N,8580,2557,111,3.325843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522145,0,24,115,12,0,0,0,1,3,...,51875.0,14821.32,3500,-13.50830,-48.7421,CO,155,113,2,0.000000
5566,522157,0,17,137,11,1,1,1,2,0,...,47960.0,16804.54,2854,-14.28400,-49.9205,CO,120,88,1,0.500000
5567,220460,0,0,14,6,2,0,1,2,1,...,30136.0,7773.10,3877,-5.98900,-42.5601,NE,230,115,1,0.000000
5568,220590,0,0,21,5,0,7,3,3,3,...,51080.0,9549.44,5349,-8.00434,-43.8723,NE,344,148,5,2.333333


Definido que somente cidades com mais de 5 registros de caso em gestantes seriam analisadas.

In [16]:
df_sif = df_sif[df_sif["sgestante"]>=5]

In [17]:
df_sif

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil,squalidade
0,110001,1,192,833,51,41,28,2,12,186,...,495629.0,21600.73,22945,-11.92870,-61.9958,N,1828,659,23,2.333333
2,110040,5,62,65,43,66,18,2,11,18,...,358464.0,16728.75,21428,-9.71470,-63.3193,N,1154,369,24,1.636364
3,110034,5,78,126,24,8,16,1,7,35,...,260348.0,18065.91,14411,-11.34670,-62.2852,N,911,414,8,2.285714
4,110002,121,252,198,224,240,296,8,89,321,...,2578830.0,23908.38,107863,-9.90612,-63.0330,N,8580,2557,111,3.325843
5,110045,9,156,721,81,98,134,6,36,105,...,726031.0,18309.15,39654,-10.20600,-63.8312,N,3064,747,39,3.722222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5525,510735,0,30,266,21,4,17,4,12,6,...,315191.0,56334.34,5595,-10.80190,-52.7351,CO,487,111,6,1.416667
5530,520085,0,11,310,14,1,9,2,10,4,...,75956.0,12429.35,6111,-16.25300,-49.9799,CO,359,207,7,0.900000
5531,520145,0,12,75,4,0,6,2,5,4,...,84493.0,34152.43,2474,-18.29450,-51.1520,CO,155,78,1,1.200000
5535,520400,0,386,153,3,2,15,1,7,35,...,257500.0,32215.68,7993,-15.80620,-46.9237,CO,529,199,8,2.142857


Definido que somente cidades com mais casos de sífilis em não gestantes do que em mulheres gestantes seriam analisados.

In [18]:
df_sif = df_sif[df_sif["sadquirida"]>df_sif["sgestante"]]

In [19]:
df_sif

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil,squalidade
0,110001,1,192,833,51,41,28,2,12,186,...,495629.0,21600.73,22945,-11.92870,-61.9958,N,1828,659,23,2.333333
2,110040,5,62,65,43,66,18,2,11,18,...,358464.0,16728.75,21428,-9.71470,-63.3193,N,1154,369,24,1.636364
3,110034,5,78,126,24,8,16,1,7,35,...,260348.0,18065.91,14411,-11.34670,-62.2852,N,911,414,8,2.285714
4,110002,121,252,198,224,240,296,8,89,321,...,2578830.0,23908.38,107863,-9.90612,-63.0330,N,8580,2557,111,3.325843
5,110045,9,156,721,81,98,134,6,36,105,...,726031.0,18309.15,39654,-10.20600,-63.8312,N,3064,747,39,3.722222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5509,500625,0,10,232,7,6,19,2,10,7,...,178414.0,46778.58,3814,-22.65570,-53.8577,CO,391,127,3,1.900000
5525,510735,0,30,266,21,4,17,4,12,6,...,315191.0,56334.34,5595,-10.80190,-52.7351,CO,487,111,6,1.416667
5531,520145,0,12,75,4,0,6,2,5,4,...,84493.0,34152.43,2474,-18.29450,-51.1520,CO,155,78,1,1.200000
5535,520400,0,386,153,3,2,15,1,7,35,...,257500.0,32215.68,7993,-15.80620,-46.9237,CO,529,199,8,2.142857


In [20]:
df_ordenado = df_sif.sort_values(by=["squalidade"], axis=0, ascending=False)
correlacao = df_ordenado[["squalidade", "sadquirida", "sgestante", "populacao", "estado",
                          "per_capita", "estabelecimentos", "profissionais"]]
correlacao.corr().style.background_gradient(axis=None)

,squalidade,sadquirida,sgestante,populacao,per_capita,estabelecimentos,profissionais
squalidade,1.000000,0.065837,0.000112,0.020008,0.176581,0.059055,0.038664
sadquirida,0.065837,1.000000,0.915144,0.953751,0.062286,0.944520,0.962007
sgestante,0.000112,0.915144,1.000000,0.949120,0.051879,0.905274,0.918934
populacao,0.020008,0.953751,0.949120,1.000000,0.066005,0.960227,0.981614
per_capita,0.176581,0.062286,0.051879,0.066005,1.000000,0.083933,0.073312
estabelecimentos,0.059055,0.944520,0.905274,0.960227,0.083933,1.000000,0.978812
profissionais,0.038664,0.962007,0.918934,0.981614,0.073312,0.978812,1.000000


Identificada a necessidade de calcular notificações e número de profissionais e estabelecimentos pela população.
Calculando número de pessoas notificadas com a doença por 100 mil habitantes nos 5 anos estudados

In [21]:
df_sif["radquirida"] = 100000*df_sif["sadquirida"]/df_sif["populacao"]
df_sif["rgestante"] = 100000*df_sif["sgestante"]/df_sif["populacao"]
df_sif["rprofissionais"] = 100000*df_sif["profissionais"]/df_sif["populacao"]
df_sif["restabelecimentos"] = 100000*df_sif["estabelecimentos"]/df_sif["populacao"]
df_sif

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil,squalidade,radquirida,rgestante,rprofissionais,restabelecimentos
0,110001,1,192,833,51,41,28,2,12,186,...,-61.9958,N,1828,659,23,2.333333,122.030944,52.298976,296.360863,152.538679
2,110040,5,62,65,43,66,18,2,11,18,...,-63.3193,N,1154,369,24,1.636364,84.002240,51.334702,121.336569,88.669031
3,110034,5,78,126,24,8,16,1,7,35,...,-62.2852,N,911,414,8,2.285714,111.026299,48.574006,208.174311,152.661162
4,110002,121,252,198,224,240,296,8,89,321,...,-63.0330,N,8580,2557,111,3.325843,274.422184,82.512076,351.371647,114.960645
5,110045,9,156,721,81,98,134,6,36,105,...,-63.8312,N,3064,747,39,3.722222,337.923034,90.785293,196.701468,83.219852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5509,500625,0,10,232,7,6,19,2,10,7,...,-53.8577,CO,391,127,3,1.900000,498.164657,262.191924,498.164657,131.095962
5525,510735,0,30,266,21,4,17,4,12,6,...,-52.7351,CO,487,111,6,1.416667,303.842717,214.477212,250.223414,178.731010
5531,520145,0,12,75,4,0,6,2,5,4,...,-51.1520,CO,155,78,1,1.200000,242.522231,202.101859,606.305578,121.261116
5535,520400,0,386,153,3,2,15,1,7,35,...,-46.9237,CO,529,199,8,2.142857,187.664206,87.576630,337.795571,100.087577


In [22]:
df_ordenado = df_sif.sort_values(by=["squalidade"], axis=0, ascending=False)
correlacao = df_ordenado[["squalidade", "radquirida", "rgestante", "rprofissionais", 
                          "restabelecimentos", "populacao", "per_capita",]]
correlacao.corr().style.background_gradient(axis=None)

,squalidade,radquirida,rgestante,rprofissionais,restabelecimentos,populacao,per_capita
squalidade,1.000000,0.560074,-0.149914,0.201578,0.270486,0.020008,0.176581
radquirida,0.560074,1.000000,0.614158,0.249113,0.222073,0.097216,0.175851
rgestante,-0.149914,0.614158,1.000000,0.093725,0.011918,0.093563,0.072282
rprofissionais,0.201578,0.249113,0.093725,1.000000,0.646760,0.186188,0.301578
restabelecimentos,0.270486,0.222073,0.011918,0.646760,1.000000,0.033484,0.231890
populacao,0.020008,0.097216,0.093563,0.186188,0.033484,1.000000,0.066005
per_capita,0.176581,0.175851,0.072282,0.301578,0.231890,0.066005,1.000000


Não existe correlação entre o número de casos confirmados de sífilis em gestantes e o número de profissionais, estabelecimentos e renda per capita de um município (<10%). Existe uma baixa correlação entre o número de casos notificados de sífilis em adultos não gestantes e o número de profissionais, estabelecimentos e renda per capita de um município (17.5 a 25%). Essa diferença não muito significativa de valores é matematicamente pequena. Porém expressa com clareza a ideia de que o sistema de notificação de sífilis em gestantes funciona relativamente bem independente do município brasileiro, enquanto a notificação de sífilis em adultos não gestantes é muito ruim. Não faz sentido acreditar que onde existem mais profissionais de saúde, mais estabelecimentos de saúde e mais riqueza, é justamente onde temos mais doenças. Acredito que esse resultado pode ser extrapolado para outros agravos e doenças.